In [56]:
import gmplot
import xml.etree.ElementTree as ET
import requests
import time

In [57]:
try:
    with open('google_maps_key.txt', 'r') as f:
        text = f.read()
    google_maps_key = "".join(text)[0:39]
except:
    google_maps_key = input()

## Get the data

Get the xml file from the RSS of the NSW RFS. Extract elements of interest and multiple polys of the areas.

Add the data to a results list.

In [58]:
xml = requests.get('http://www.rfs.nsw.gov.au/feeds/majorIncidents.xml').text

filetime = time.strftime('%b-%d-%Y_%H%M', time.localtime())

with open(f"majorIncidents-{filetime}.txt", 'w') as f:
    f.write(xml)

root = ET.fromstring(xml)
results = []
for child in root:
    for ch0 in child:
        if ch0.tag == 'item':
            
            poly = []
            for ch1 in ch0:
                #print(ch1.tag, ch1.text)
                if ch1.tag == 'title':
                    title = ch1.text
                if ch1.tag == 'pubDate':
                    pubdate = ch1.text
                if ch1.tag == 'category':
                    category = ch1.text
                if ch1.tag == '{http://www.georss.org/georss}point':
                    point = ch1.text
                if ch1.tag == "{http://www.georss.org/georss}polygon":
                    poly.append(ch1.text)
    
            results.append([title,pubdate,category,poly,point])
    

In [59]:
print(f"There are {len(results)} areas of interest.")

There are 79 areas of interest.


In [68]:
print(f"The number of Emergency Warnings is: {sum([item[2] == 'Emergency Warning' for item in results])}")

The number of Emergency Warnings is: 0


In [60]:
# Get the centroid of all of the points for the events. This is to set the starting point for the map.
_ = []

# For the long and the lat get the average
for i in range(0,2):
    _.append([float(y[i]) for y in [ x[4].split(' ') for x in results]])

centroid_long_lat = [sum(x) / len(x) for x in _]
print(centroid_long_lat)

[-34.196428171822674, 150.09933012954335]


In [61]:
colours = {'Emergency Warning': 'red', 'Watch and Act' : 'orange', 'Advice' : 'blue'}

In [62]:
gmap = gmplot.GoogleMapPlotter(centroid_long_lat[0], centroid_long_lat[1], 8)

gmap.apikey = google_maps_key

for item in results:
    
    # convert strings to list of floats.
    
    # start
    polys = [

    [[float(z) for z in y.split(' ')[::2]], [float(z) for z in y.split(' ')[::-2]]]

    for y in item[3]]
    # end
    
    for poly in polys:
        gmap.polygon(poly[0], poly[1], color = colours[item[2]])

mapname = f"map-{filetime}.html"
gmap.draw(mapname)

In [63]:
from IPython.display import IFrame

IFrame(src=mapname, width=700, height=600)